# Análisis exploratorio de incidencia delictiva en México

## Descarga los datos
Los datos están limpios y listos para descargar [en esta liga](https://drive.google.com/file/d/1yjeD7lH_rhs86xn0XI0fTQtc-9TYNvnA/view?usp=sharing)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df = pd.read_csv('data/delitos.csv')

In [ ]:
df.head()

In [ ]:
df.tipo_delito.unique()

#### Ejercicio 1: Elige 3 estados de la república y grafica una serie de tiempo de la frecuencia abosluta de homicidios dolosos de enero 2015 a julio 2019 en estas tres entidades

In [ ]:
#Oaxaca, CDMX, Puebla
df_geo = pd.DataFrame({
    'clave_entidad': df.clave_entidad.unique(),
    'entidad': df.entidad.unique()
})
df_geo.head()

In [ ]:
#entidades seleccionadoas
df_geo[(df_geo.entidad == 'Oaxaca') | (df_geo.entidad == 'Puebla') | (df_geo.entidad == 'Ciudad de México')]

In [ ]:
homicidios_dolosos = df[df.subtipo_delito == 'Homicidio doloso']
homicidios_dolosos.head()

In [ ]:
homicidios_dolosos = homicidios_dolosos[(homicidios_dolosos.clave_entidad == 9) | (homicidios_dolosos.clave_entidad == 20) | (homicidios_dolosos.clave_entidad == 21)]
homicidios_dolosos.head()

In [ ]:
agregacion_entidad_homicidios = homicidios_dolosos.groupby(['anio','mes', 'clave_entidad', 'entidad']) \
                                                  .agg({'frecuencia': 'sum'}) \
                                                  .reset_index()

agregacion_entidad_homicidios['aniomes'] = agregacion_entidad_homicidios.apply(lambda row: int(str(row['anio']) + str(row['mes']).zfill(2)), axis=1)
agregacion_entidad_homicidios.head()

In [ ]:
hom_cdmx = agregacion_entidad_homicidios[agregacion_entidad_homicidios.clave_entidad == 9]
hom_oax = agregacion_entidad_homicidios[agregacion_entidad_homicidios.clave_entidad == 20]
hom_pue = agregacion_entidad_homicidios[agregacion_entidad_homicidios.clave_entidad == 21]

In [ ]:
hom_cdmx.tail()

In [ ]:
plt.figure(figsize=(14,7))
plt.plot(range(len(hom_cdmx.aniomes)), hom_cdmx.frecuencia,'b-' ,label="CDMX" )
plt.plot(range(len(hom_oax.aniomes)), hom_oax.frecuencia,'r-' ,label="Oaxaca" )
plt.plot(range(len(hom_pue.aniomes)), hom_pue.frecuencia,'g-' ,label="Puebla" )
plt.xticks(range(len(hom_cdmx.aniomes)), hom_cdmx.aniomes, rotation=90)
plt.legend(loc='best')
plt.show()

#### Ejercicio 2: Contetas las siguientes  preguntas:
1. ¿Cuántos homicidios dolosos hubo en Colima en el 2018?
2. ¿Cuantos robos de vehículo automotor ha habido en el 2019?
3. Obten la suma de homicidos dolosos y feminidios en toda la República Mexicana en cada año.
4. ¿En qué mes y en qué municipio ha ocurrido el mayor número de feminicidios?
5. ¿En qué año y en qué estado ha ocurrido el mayor número de feminicidios?

#### Ejercicio 3: Haz una gráfica de pastel de tipos de delito. Deberás crear una gráfica para cada año. Utilzia la función subplots de matplotlib

In [ ]:
df3 = df.groupby(['anio', 'tipo_delito']).agg({'frecuencia':'sum'}).reset_index()
plt.figure(figsize=(14,14))
plt.pie(df3[df3.anio == 2015].frecuencia, labels=df3[df3.anio == 2015].tipo_delito)
plt.tight_layout()
plt.show()

---
## Tasa por 100,000 habitantes
Mostrar el total de delitos en una entidad no nos sirve de mucho. Es mucho más útil calcular la tasa de incidencia delictiva por cada 100,000 habitantes

$$
tasa = \frac{delitos\space totales}{población} \times 100,000
$$

Esta tasa la podemos anualizar multiplicándola por un factor de 12
$$
tasa\space anualizada = tasa \times 12
$$

### Población por entidad federativa según [la encuesta intercensal 2015](https://www.inegi.org.mx/programas/intercensal/2015/)

---

In [ ]:
pobs = pd.read_csv('data/poblacion_entidades_2015.csv', encoding='iso-8859-1', sep=";")
pobs = pobs[['Cve_Entidad', 'Entidad', 'Poblacion']]
pobs = pobs.rename(columns={'Cve_Entidad':'clave_entidad', 'Entidad':'entidad', 'Poblacion':'poblacion'})
pobs.head()

In [ ]:
df3 = df.groupby(['anio', 'tipo_delito']).agg({'frecuencia':'sum'}).reset_index()
df3.groupby(['tipo_delito']).agg({'frecuencia':'sum'}).reset_index().sort_values(['frecuencia']).tipo_delito.values